# Connecting mocks to spectra.

This notebook presents QA checks of the code which assigns spectra to the mock catalogs.

### Initialize

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import os
import numpy as np
import warnings
import yaml
from glob import glob

## Visualize the surface density of the input mocks.

In [3]:
# Note that you need the standard desi/conda dependencies + matplotlib, 
# basemap, and healpy to run this notebook.
import mpl_toolkits
import matplotlib.pyplot as plt

In [4]:
from desiutil.plots import init_sky, plot_sky_binned

In [5]:
def _GRFpath():
    return os.path.join(os.getenv('DESI_ROOT'), 'mocks', 'GaussianRandomField', 'v0.0.5')

In [6]:
def read_GRF(fitsfile):
    """Read the Gaussian Random Field mocks."""
    import fitsio
    from astropy.table import Table, Column
    print('Reading {}'.format(fitsfile))
    dat = Table(fitsio.read(fitsfile, ext=1, upper=True))
    if 'random' not in fitsfile:
        dat.add_column(Column(name='Z', data=(dat['Z_COSMO'] + dat['DZ_RSD']).astype('f4')))
    return dat

In [7]:
def read_MXXL(hdf5file):
    """Read the MXXL/BGS mock."""
    import h5py
    print('Reading {}'.format(hdf5file))
    f = h5py.File(hdf5file)
    ra  = f['Data/ra'][...].astype('f8') % 360.0
    dec = f['Data/dec'][...].astype('f8')
    zobs = f['Data/z_obs'][...].astype('f4')
    rmag = f['Data/app_mag'][...].astype('f8')
    these = rmag < 20
    dat = {'RA': ra[these], 'DEC': dec[these], 'Z': zobs[these]}
    return dat

In [ ]:
def qamock_sky(cat, objtype, ax=None):
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        basemap = init_sky(galactic_plane_color='k', ax=ax[0]);
        plot_sky_binned(cat['RA'], cat['DEC'], verbose=False, 
                        clip_lo='!1', plot_type='healpix', cmap='jet', 
                        label=r'{} (targets/deg$^2$)'.format(objtype), 
                        basemap=basemap)
    if 'Z' in cat.keys():
        ax[1].hist(cat['Z'])

In [ ]:
data = list()
print(r'Note: the BGS mock is selected to have $r_{SDSS}<20$, which is not quite right.')
mockfile = os.path.join(os.getenv('DESI_ROOT'), 'mocks', 'bgs', 'MXXL', 'desi_footprint', 'v0.0.3', 'BGS.hdf5')
data.append(read_MXXL(mockfile))

for mock in ('ELG', 'LRG', 'QSO'):
    mockfile = os.path.join(_GRFpath(), '{}.fits'.format(mock))
    data.append(read_GRF(mockfile))

mockfile = os.path.join(os.getenv('DESI_ROOT'), 'mocks', 'GaussianRandomField', 'v0.0.1', '2048', 'random.fits')
data.append(read_GRF(mockfile))

In [ ]:
# Why doesn't this work?!?
for cat, objtype in zip(data, ('BGS', 'ELG', 'LRG', 'QSO', 'SKY')):
#for cat, objtype in zip(data, np.atleast_1d('BGS')):
    qamock_sky(cat, objtype)

## Assess the QSO mock targets, spectra, and contaminants.

In [ ]:
from desitarget.mock.build import targets_truth

In [ ]:
two_percent_path = os.path.join(os.getenv('DESI_ROOT'), 'datachallenge', 'two_percent')

### Generate a QSO target catalog.

In [ ]:
def target_yaml(target='QSO', bounds=(203, 204, 0, 1), outfile='mock_input.yaml', verbose=False):
    """Write a target yaml file on-the-fly for various combinations of input parameters."""
    print('Writing {}'.format(outfile))
    yf = open(outfile, 'w')
    yf.write('dust_dir: {}\n'.format(os.path.join(os.getenv('DUST_DIR'), 'maps')))
    yf.write('decals_brick_info: {}\n'.format(os.path.join(os.getenv('DESI_ROOT'), 'target', 
                                                           'catalogs', 'brick-info-dr3.v0.0.1.fits')))
    yf.write('subset:\n')
    yf.write('    ra_dec_cut: True\n')
    for blabel, bbound in zip(('min_ra', 'max_ra', 'min_dec', 'max_dec'), bounds):
        yf.write('    {}: {:g}\n'.format(blabel, bbound))
    yf.write('sources:\n')
    for targ in np.atleast_1d(target):
        yf.write('    {}: {{\n'.format(targ.upper()))
        yf.write('        target_name: {},\n'.format(targ.upper()))
        if targ == 'QSO':
            yf.write('        mock_dir_name: {},\n'.format(_GRFpath()))
            yf.write('        format: gaussianfield,\n')
            yf.write('        density: 170\n')
        yf.write('    }\n')
    yf.close()    
    if verbose:
        print()
        with(open(outfile, 'r')) as yf:
            print(yf.read())

In [ ]:
config_file = os.path.join(two_percent_path, 'qso_input.yaml')
target_yaml(target='QSO', outfile=config_file, verbose=True)

In [ ]:
output_dir = os.path.join(two_percent_path, 'qso_targets')
bricksize = 0.25
seed = 555
nproc = 4
with open(config_file, 'r') as pfile:
    params = yaml.load(pfile)
targets_truth(params, output_dir, realtargets=None, seed=seed, verbose=True, 
              nproc=nproc, bricksize=bricksize, outbricksize=bricksize)

### Read the catalog back in and make some plots.